# Task 1: MNIST + OOP

This notebook demonstrates an object‐oriented approach for MNIST classification using three models:
1. Random Forest
2. Feed-Forward Neural Network
3. Convolutional Neural Network (CNN)

I'm choosing to use a Jupyter notebook because Docker and VS Code environments have caused persistent dependency and configuration issues, leading to long build times and debugging headaches. Given the one-week deadline, Jupyter offers a ready-to-use, minimal-overhead environment that lets me focus on coding and quickly iterate without the extra complexity.

## 1. Task 1: MNIST + OOP

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from abc import ABC, abstractmethod
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pytorch_lightning as pl

pl.seed_everything(42)  # For reproducibility

c:\Users\maksy\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 42


42

### 1.1. Define the Interface

In [2]:
class MnistClassifierInterface(ABC):
    """
    Interface requiring two abstract methods:
     - train(train_data, train_targets)
     - predict(test_data)
    """
    @abstractmethod
    def train(self, train_data, train_targets):
        pass

    @abstractmethod
    def predict(self, test_data):
        pass

### 1.2. Random Forest Classifier

In [3]:
class RfMnistClassifier(MnistClassifierInterface):
    def __init__(self, n_estimators=100):
        self.model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

    def train(self, train_data, train_targets):
        self.model.fit(train_data, train_targets)

    def predict(self, test_data):
        return self.model.predict(test_data)

### 1.3. Feed-Forward NN

In [4]:
class FeedForwardMnist(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.layers(x)

class NnMnistClassifier(MnistClassifierInterface):
    def __init__(self, epochs=5, lr=1e-3, batch_size=64):
        self.epochs = epochs
        self.lr = lr
        self.batch_size = batch_size
        self.model = FeedForwardMnist()

    def train(self, train_data, train_targets):
        X = torch.from_numpy(train_data).float()
        y = torch.from_numpy(train_targets).long()
        dataset = torch.utils.data.TensorDataset(X, y)
        loader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()

        self.model.train()
        for _ in range(self.epochs):
            for batch_x, batch_y in loader:
                optimizer.zero_grad()
                out = self.model(batch_x)
                loss = criterion(out, batch_y)
                loss.backward()
                optimizer.step()

    def predict(self, test_data):
        X = torch.from_numpy(test_data).float()
        self.model.eval()
        with torch.no_grad():
            out = self.model(X)
            preds = torch.argmax(out, dim=1)
        return preds.numpy()

### 1.4. CNN Classifier

In [5]:
class ConvMnistNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

class CnnMnistClassifier(MnistClassifierInterface):
    def __init__(self, epochs=5, lr=1e-3, batch_size=64):
        self.epochs = epochs
        self.lr = lr
        self.batch_size = batch_size
        self.model = ConvMnistNet()
        
    def train(self, train_data, train_targets):
        X = torch.from_numpy(train_data).float().view(-1,1,28,28)
        y = torch.from_numpy(train_targets).long()
        dataset = torch.utils.data.TensorDataset(X, y)
        loader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()

        self.model.train()
        for _ in range(self.epochs):
            for batch_x, batch_y in loader:
                optimizer.zero_grad()
                out = self.model(batch_x)
                loss = criterion(out, batch_y)
                loss.backward()
                optimizer.step()

    def predict(self, test_data):
        X = torch.from_numpy(test_data).float().view(-1,1,28,28)
        self.model.eval()
        with torch.no_grad():
            out = self.model(X)
            preds = torch.argmax(out, dim=1)
        return preds.numpy()

### 1.5. Wrapper Class

In [6]:
class MnistClassifier:
    def __init__(self, algorithm='rf'):
        if algorithm == 'rf':
            self.impl = RfMnistClassifier()
        elif algorithm == 'nn':
            self.impl = NnMnistClassifier()
        elif algorithm == 'cnn':
            self.impl = CnnMnistClassifier()
        else:
            raise ValueError(f"Unsupported algorithm: {algorithm}")

    def train(self, train_data, train_targets):
        self.impl.train(train_data, train_targets)

    def predict(self, test_data):
        return self.impl.predict(test_data)

### 1.6. Demo on MNIST

In [7]:
# Load MNIST and do a small training demo
mnist_transform = transforms.Compose([
    transforms.ToTensor()
])

train_mnist = torchvision.datasets.MNIST(root='./mnist_data', train=True, download=True, transform=mnist_transform)
test_mnist = torchvision.datasets.MNIST(root='./mnist_data', train=False, download=True, transform=mnist_transform)

# We'll just take 5k from the training set to keep it quick
train_small, _ = random_split(train_mnist, [5000, len(train_mnist)-5000])

def dataset_to_numpy(dataset):
    images, labels = [], []
    for img, label in dataset:
        # Convert to numpy array
        np_img = np.array(img).astype(np.float32)
        images.append(np_img)
        labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# Convert
X_train_mnist, y_train_mnist = dataset_to_numpy(train_small)
X_test_mnist, y_test_mnist = dataset_to_numpy(test_mnist)

# Flatten for rf/nn
X_train_flat = X_train_mnist.reshape(-1, 28*28)
X_test_flat = X_test_mnist.reshape(-1, 28*28)

# 1) Random Forest
rf_model = MnistClassifier(algorithm='rf')
rf_model.train(X_train_flat, y_train_mnist)
rf_preds = rf_model.predict(X_test_flat)
rf_acc = accuracy_score(y_test_mnist, rf_preds)
print("Random Forest Accuracy:", rf_acc)

# 2) Feed-forward NN
nn_model = MnistClassifier(algorithm='nn')
nn_model.train(X_train_flat, y_train_mnist)
nn_preds = nn_model.predict(X_test_flat)
nn_acc = accuracy_score(y_test_mnist, nn_preds)
print("Feed-Forward NN Accuracy:", nn_acc)

# 3) CNN
cnn_model = MnistClassifier(algorithm='cnn')
cnn_model.train(X_train_flat, y_train_mnist)
cnn_preds = cnn_model.predict(X_test_flat)
cnn_acc = accuracy_score(y_test_mnist, cnn_preds)
print("CNN Accuracy:", cnn_acc)

Random Forest Accuracy: 0.9443
Feed-Forward NN Accuracy: 0.9261
CNN Accuracy: 0.9628


## Conclusion
We've shown an object-oriented approach for MNIST classification using three models (Random Forest, Feed-Forward NN, and CNN).